In [1]:
from nltk.corpus import treebank,brown, movie_reviews, wordnet
from gensim.models import Word2Vec
import gensim.models
import pandas as pd
import numpy as np
from nltk.data import find

In [2]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [4]:
# CHANGE THIS TO YOUR INPUT FILE PATH
# SHOULD BE A CVS FILE FROM PREVIOUS STEP\n ,
input_fname = 'data/amr-bank-struct-v1.6-training.csv'

# CHANGE THIS TO YOUR OUTPUT FILE PATH\n ,
output_fname = 'train' 

# if you want S * 5 * 300
# Change this to five
# if you want padded input
# change this to pad
mode = 'pad'

In [5]:
df = pd.read_csv(input_fname) 
# df = pd.read_csv('data/1.csv') 
nor_word = list(df['normalized_words']) 
isfocus = list(df['isfocus']) 
index = list(df['index']) 

In [14]:
def prepro(): 
    first = True 
    max_length = 0 
    for i in range(len(index)): 
        if index[i]== 0 or i+1 == len(index): 
            if first: 
                first = False 
            else: 
                if len(tmp_nword)> max_length: 
                    max_length = len(tmp_nword) 
                wv = [] 
                sent.append(tmp_nword) 
                for j in tmp_nword: 
                    if j in model: 
                        wv.append(model[j]) 
                    else: 
                        wv.append(np.zeros(300)) 
                assert(len(wv) == index[i-1]+1) 
                n.append(np.asarray(wv)) 
                assert(len(tmp_f) == len(wv)) 
                f.append(np.asarray(tmp_f)) 
            tmp_nword = [] 
            tmp_f = [] 
            
        tmp_nword.append(nor_word[i]) 
        tmp_f.append(isfocus[i]) 
    return max_length 



In [15]:
def padding(n,f): 
    for i in range(len(n)): 
        if len(n[i]) < max_length: 
            pad_num = max_length - len(n[i]) 
            n[i] = np.concatenate((n[i],np.empty((pad_num,300))),axis=0) 
            
    # 0 for no-word 
    # 1 for non-focus 
    # 2 for focus 
    for i in range(len(f)): 
        for j in range(len(f[i])): 
            if f[i][j] == 1: 
                f[i][j] = 2 

            if f[i][j] == 0: 
                f[i][j] = 1 

    for i in range(len(f)): 
        if len(f[i]) < max_length: 
            pad_num = max_length - len(f[i]) 
            f[i] = np.concatenate((f[i],np.zeros(pad_num)),axis=0)  
            
    return n,f

In [16]:
def div_5(n,f):
    n5 = []
    f5 = []
    for i in range(len(n)):
        if len(n[i]) < 5: 
            tmp = n[i] 
            tmp = np.concatenate((tmp,np.zeros((5-len(n[i]),300)))) 
            n5.append(np.array(tmp)) 

            tmpf = f[i] 
            tmpf = np.concatenate((tmpf,np.zeros(5-len(f[i])))) 
            f5.append(np.array(tmpf)) 

        for j in range(len(n[i])-4): 
            n5.append(n[i][j:j+5]) 
            f5.append(f[i][j:j+5]) 

    return n5,f5 

In [17]:
n = [] 
f = [] 
sent = [] 
max_length = prepro() 
assert(len(n) == len(f)) 

In [18]:
if mode == 'pad':
    nn,ff = padding(n,f)
else:
    nn,ff = div_f(n,f)

nn = np.asarray(nn)
ff = np.asarray(ff)
    

In [19]:
if mode == 'five':
    np.save('word_embedding_divby5_'+output_fname,nn)
    np.save('isfocus_divby5_'+output_fname,ff)
else:
    np.save('word_embedding_'+ output_fname,nn)
    np.save('isfocus_'+ output_fname,ff)

In [20]:
nn.shape

(1274, 101, 300)

In [21]:
ff.shape

(1274, 101)